In [1]:
import pickle
import os
import torch

In [23]:
with open('d:\\year 4\\semester 1\\BT\\BT 4033\\prediction\\GO terms\\chains and cc_go terms.pkl', 'rb') as f:
    chains_n_cc = pickle.load(f)    # capital ids 

In [24]:
pdb_n_cc = dict()
for k, v in chains_n_cc.items():
    pdb_n_cc[k.split('_')[0]] = v   # capital ids 

In [4]:
with open('d:\\year 4\\semester 1\\BT\\BT 4033\\prediction\\labels\\pdb_uniprot_map_dict.pkl', 'rb') as f:
    pdb_uni_map = pickle.load(f)    # simple pdb, capital uni

In [5]:
with open('d:\\year 4\\semester 1\\BT\\BT 4033\\ppi_part\\ppi_vgae_embedds_dict', 'rb') as f:
    ppi_embedd = pickle.load(f) # capital uni ids 

In [6]:
# Set the folder path and the number of files to select
pooledmap_path = 'D:\\year 4\\semester 1\\BT\\BT 4033\\structure and seq\\pooled maps\\'

# Get the list of all files in the folder
all_files = os.listdir(pooledmap_path)

# Filter out any non-files (like directories)
all_pool_files = [f for f in all_files if os.path.isfile(os.path.join(pooledmap_path, f))]

In [25]:
dataset = dict()
for pool in all_pool_files:
    pdb_id = pool.split('_')[0]
    try:
        go_terms = pdb_n_cc[pdb_id.upper()]
        uniprot_id = pdb_uni_map[pdb_id]
        ppi_vect = ppi_embedd[uniprot_id]
        pooled = torch.load(pooledmap_path+pool).tolist()
        concat = pooled + ppi_vect
        dataset[pool.split('.')[0]] = {'terms': go_terms,
                           'vect': concat}
    except:
        continue

In [27]:
len(dataset)

13813

In [28]:
init_label_counts = dict()
for v in dataset.values():
    for term in v['terms']:
        if term in init_label_counts.keys():
            init_label_counts[term] += 1
        else:
            init_label_counts[term] = 1

In [29]:
cc_terms_over_50 = set(k for k, v in init_label_counts.items() if v >= 50)
len(cc_terms_over_50)

76

In [30]:
filterd_labels_and_inputs = dict()
for k, v in dataset.items():
    label_intersect = set(v['terms']).intersection(cc_terms_over_50)
    if len(label_intersect) != 0:
        filterd_labels_and_inputs[k] = {'terms': list(label_intersect),
                                        'vect': v['vect']}
len(filterd_labels_and_inputs)

13753

In [31]:
labels_set = [v['terms'] for v in filterd_labels_and_inputs.values()]

In [32]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
binary_labels = mlb.fit_transform(labels_set)

In [33]:
binary_labels.shape

(13753, 76)

In [54]:
mlb.classes_

array(['GO:0005634', 'GO:0005737', 'GO:0005829', 'GO:0005886'],
      dtype=object)

In [34]:
binary_labels

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [35]:
with open('cc_terms_class_order.pkl', 'wb') as f:
    pickle.dump(mlb.classes_.tolist(), f)

In [36]:
i = 0
for v in filterd_labels_and_inputs.values():
    v['enc_label'] = binary_labels[i].tolist()
    i += 1

In [37]:
with open('cc_dataset.pkl', 'wb') as f:
    pickle.dump(filterd_labels_and_inputs, f)